In [2]:
import common.ipynb_importer
# from db.pg.pg_00_common import *
import common.pg_common

cursor = connect()

NameError: name 'connect' is not defined

# Trigger

触发器就是一个自动调用函数，当一个特定的数据库事件发生时，触发器会被自动执行。数据库事件包括：
- insert
- update
- delete
- truncate

触发器可以在表上定义，当表上的事件发生时，触发器会被调用。触发器可以在行级别或者语句级别触发。

## 触发器类型
- Row-level trigger: 当行被插入、更新或者删除时触发
- Statement-level trigger: 当SQL语句执行时触发

两种类型之间的区别在于调用触发器的次数和时间。

## 何时使用触发器
希望在数据库中保持跨功能，只要表的数据被修改，触发器就会自动运行。同时，可以用触发器去维护复杂数据的一致性。例如当 A 表的数据被修改时，B 表的数据也要被修改。
缺点是必须知道触发器的存在，并了解其逻辑，才能知道数据变化时的效果。

## PostgreSQL 触发器 VS SQL 标准触发器
PostgreSQL 触发器与 SQL 标准触发器有一些不同之处：
- PostgreSQL 为 `TRUNCATE` 事件会触发触发器；
- PostgreSQL 允许在视图上定义触发器；
- PostgreSQL 要求定义一个用户定义的函数作为触发器的操作。


# 创建触发器

步骤：
1. 使用 `CREATE FUNCTION` 语句创建一个触发器函数
2. 使用 `CREATE TRIGGER` 语句将表绑定到触发器函数上

语法
```
CREATE FUNCTION trigger_function() 
   RETURNS TRIGGER 
   LANGUAGE PLPGSQL
AS $$
BEGIN
   -- trigger logic
END;
$$
``` 

触发器函数可以通过一个称之为 `TriggerData` 的特殊数据结构来接收调用上下文的数据，这个 `TriggerData` 结构包含了触发器的事件类型、触发器的表、触发器的行等信息。例如，`OLD` 和 `NEW` 变量分别代表了触发器的旧行和新行。PostgreSQL 同时还提供了一些以 `TG_` 开头的局部变量，例如：`TG_NAME`、`TG_WHEN`、`TG_LEVEL`、`TG_OP`、`TG_RELID`、`TG_TABLE_NAME`、`TG_TABLE_SCHEMA`、`TG_NARGS`、`TG_ARGV` 等。

创建好触发器函数后，可以将其绑定到多个触发器时间，如：`BEFORE INSERT`、`AFTER UPDATE`、`BEFORE DELETE` 等。

## CREATE TRIGGER 语句

语法
```
CREATE TRIGGER trigger_name 
   {BEFORE | AFTER} { event }
   ON table_name
   [FOR [EACH] { ROW | STATEMENT }]
       EXECUTE PROCEDURE trigger_function
```

In [4]:
sql = """
CREATE TABLE employees(
   id INT GENERATED ALWAYS AS IDENTITY,
   first_name VARCHAR(40) NOT NULL,
   last_name VARCHAR(40) NOT NULL,
   PRIMARY KEY(id)
);

CREATE TABLE employee_audits (
   id INT GENERATED ALWAYS AS IDENTITY,
   employee_id INT NOT NULL,
   last_name VARCHAR(40) NOT NULL,
   changed_on TIMESTAMP NOT NULL
);

CREATE OR REPLACE FUNCTION log_last_name_changes()
  RETURNS TRIGGER 
  LANGUAGE PLPGSQL
  AS
$$
BEGIN
	IF NEW.last_name <> OLD.last_name THEN
		 INSERT INTO employee_audits(employee_id,last_name,changed_on)
		 VALUES(OLD.id,OLD.last_name,now());
	END IF;

	RETURN NEW;
END;
$$
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1b5bcd6e610>

In [6]:
trigger_sql = """
CREATE TRIGGER last_name_changes
  BEFORE UPDATE
  ON employees
  FOR EACH ROW
  EXECUTE PROCEDURE log_last_name_changes();
"""
cursor.execute(trigger_sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1b5bcd6e610>

In [9]:
sql = """
INSERT INTO employees (first_name, last_name)
VALUES ('John', 'Doe');

INSERT INTO employees (first_name, last_name)
VALUES ('Lily', 'Bush');
"""
cursor.execute(sql)

query = """
SELECT * FROM employees;
"""
run_sql(cursor, query)

   id first_name last_name
0   1       John       Doe
1   2       Lily      Bush
2   3       John       Doe
3   4       Lily      Bush
4   5       John       Doe
5   6       Lily      Bush
6   7       John       Doe
7   8       Lily      Bush


In [10]:
sql = """
UPDATE employees
SET last_name = 'Brown'
WHERE ID = 2;
"""
cursor.execute(sql)

   id first_name last_name
0   1       John       Doe
1   3       John       Doe
2   4       Lily      Bush
3   5       John       Doe
4   6       Lily      Bush
5   7       John       Doe
6   8       Lily      Bush
7   2       Lily     Brown


In [11]:
query = """
SELECT * FROM employee_audits;
"""
run_sql(cursor, query)

   id  employee_id last_name                 changed_on
0   1            2      Bush 2024-04-10 08:21:43.149903


# 删除触发器

语法
```
DROP TRIGGER [IF EXISTS] trigger_name 
ON table_name 
[ CASCADE | RESTRICT ];
```

In [12]:
sql = """
CREATE FUNCTION check_staff_user()
    RETURNS TRIGGER
AS $$
BEGIN
    IF length(NEW.username) < 8 OR NEW.username IS NULL THEN
        RAISE EXCEPTION 'The username cannot be less than 8 characters';
    END IF;
    IF NEW.NAME IS NULL THEN
        RAISE EXCEPTION 'Username cannot be NULL';
    END IF;
    RETURN NEW;
END;
$$
LANGUAGE plpgsql;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1b5bcd6e610>

In [13]:
sql = """
CREATE TRIGGER username_check 
    BEFORE INSERT OR UPDATE
ON staff
FOR EACH ROW 
    EXECUTE PROCEDURE check_staff_user();
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1b5bcd6e610>

In [14]:
sql = """
DROP TRIGGER username_check
ON staff;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1b5bcd6e610>

# 修改触发器

语法
```
ALTER TRIGGER trigger_name
ON table_name 
RENAME TO new_trigger_name;
```

PostgreSQL 不支持修改触发器的定义，只能删除触发器并重新创建，也就是使用 `DROP TRIGGER` 和 `CREATE TRIGGER` 语句。

```
BEGIN;

DROP TRIGGER IF EXISTS salary_before_update 
ON employees;

CREATE TRIGGER salary_before_udpate
  BEFORE UPDATE
  ON employees
  FOR EACH ROW
  EXECUTE PROCEDURE validate_salary();

COMMIT;
```